# SMS Spammer (Калинка Георгий 14321 ПИ)

## Импорт библиотек

In [13]:
import re
import nltk
import gzip
import pickle
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer


In [ ]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/xitowzys/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
data = gzip.open("./data/SMSSpamCollection.txt.train.gz", "rb")
text = data.read()
data.close()
file = open('./data/SMSSpamCollection.txt.train', 'wb')
file.write(text)
file.close()


## 1. Просмотр данных через Pandas

---



In [ ]:
messages = pd.read_csv("./data/SMSSpamCollection.txt.train", sep='\t', names=["label", "message"])


In [ ]:
messages.head()


,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
messages.groupby('label').describe()


message                                                               
        count unique                                                top freq
label                                                                       
ham      4327   4068                             Sorry, I'll call later   26
spam      673    593  Please call our customer service representativ...    4

## 2. Преобразуем данные

In [ ]:
def classifier_bag_words(mess):
    messages_no_punctuation = []
    messages_classifier = []

    # Удалить ненужные символы, цифры и знаки препинания
    for line in mess:
        if line:
            messages_no_punctuation.append(re.sub('[^A-Za-z]', ' ', line))

    # Соединяем символы
    messages_no_punctuation = ''.join(messages_no_punctuation)

    # Делаем слова строчными
    messages_no_punctuation = messages_no_punctuation.lower()

    # Удаляем стоп-слова из списка
    for word in messages_no_punctuation.split():
        messages_classifier.append(word)

    for word in messages_classifier:
        if word in nltk.corpus.stopwords.words('english'):
            messages_classifier.remove(word)

    return messages_classifier


In [ ]:
# Проверка classifier_bag_words
messages['message'].head().apply(classifier_bag_words)


0    [go, jurong, point, crazy, available, in, bugi...
1                       [ok, lar, joking, wif, u, oni]
2    [free, entry, a, wkly, comp, win, fa, cup, fin...
3        [u, dun, say, early, hor, u, c, already, say]
4    [nah, don, think, goes, usf, lives, around, th...
Name: message, dtype: object

## 3. Обучение

In [86]:
X_train, x_test, Y_train, y_test = train_test_split(
    messages['message'], messages['label'], test_size=0.2)
print(len(X_train), len(x_test), len(X_train) + len(x_test))


pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=classifier_bag_words)),
    ('tidf', TfidfTransformer()),
    ('classifier', ComplementNB())
])

pipeline.fit(X_train, Y_train)


4000 1000 5000


Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function classifier_bag_words at 0x7ff051672040>)),
                ('tidf', TfidfTransformer()), ('classifier', ComplementNB())])

In [87]:
predictions = pipeline.predict(x_test)


## 4. Результат работы нейронной сети

In [88]:
print('Accuracy: ', format(accuracy_score(y_test, predictions)))


Accuracy:  0.971


## 5. Сохранение нейронной сети

In [15]:
# save the model to disk
filename = './models/finalized_model.obj'
pickle.dump(pipeline, open(filename, 'wb'))